In [2]:
# Import required modules
from langchain_ollama import OllamaLLM
from langgraph.prebuilt import create_react_agent
from langchain.schema import SystemMessage, HumanMessage
from langchain_community.agent_toolkits.openapi.toolkit import RequestsToolkit
from langchain_community.utilities.requests import TextRequestsWrapper
from app.utils.config import config  # Import the pedantic-based config object



# Fetch the configuration variables from the config object
base_url = config.ALPHAVANTAGE_BASE_URL
api_key = config.ALPHAVANTAGE_API_KEY

# Load API documentation from the `api_doc.txt` file
with open("api_doc.txt", "r") as file:
    api_spec = file.read()

# Define the system prompt with dynamic variables
system_prompt_agente_agnostico = f"""
Tienes acceso a una API para responder a las consultas que te hagan.
- Se te proveerá la documentación de la API.
- Debes utilizar las herramientas proporcionadas para consultar la API y responder a la pregunta del usuario.
Aquí está la documentación de la API:

Documentación de la API:
{api_spec}

Detalles adicionales:
- Base URL: {base_url}
- API Key: {api_key}

Cuando necesites hacer una solicitud GET, asegúrate de incluir el API Key como parte de los parámetros.
"""

# Initialize the language model (LLM)
llm = OllamaLLM(model="mistral")  # Replace with the desired model
slm = OllamaLLM(model="smollm")

# Create tools for interacting with the API (replace this with actual tools for your API)

toolkit = RequestsToolkit(
    requests_wrapper=TextRequestsWrapper(headers={}),
    allow_dangerous_requests=True,
)

tools = toolkit.get_tools()

tools

# Initialize the agent
agente_agnostico = create_react_agent(
    model=llm,
    tools=tools,
    state_modifier=SystemMessage(content=system_prompt_agente_agnostico),
)

# Define the user question
user_question = "quiero saber el precio del barril de brent (BRENT) de manera mensual para enero de 2025"

# Query the agent and stream responses
print("=== Prueba del Agente Agnóstico ===")
for chunk in agente_agnostico.stream(
    {
        "messages": [
            SystemMessage(content=system_prompt_agente_agnostico),
            HumanMessage(content=user_question),
        ]
    }
):
    print(chunk)
    print("----")


AttributeError: 'OllamaLLM' object has no attribute 'bind_tools'

In [5]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from typing import Dict
from app.utils.logger import get_logger
from app.utils.datatypes import (
    UserInputString,
    EntityExtractOutput,
    DataFetchOutput,
    FinalPredictionState,
    StockPredictionRequest,
    StockPredictionOutput
)
from app.graph_nodes.stock.stock_extractor_agent import StockDataExtractor
from app.graph_nodes.stock.stock_fetch_data import HistoricalDataFetcher
from app.graph_nodes.stock.model_training import StockDataTrainer
from app.graph_nodes.stock.prediction import StockPredictor


# **✅ Initialize LangGraph workflow**
graph = StateGraph(UserInputString)

# **✅ Logger setup**
logger = get_logger("StockPredictionWorkflow")

# **✅ Step 1: Entity Extraction Node**
def entity_extraction_node(state: UserInputString) -> EntityExtractOutput:
    logger.info("Running Entity Extraction Node...")
    extractor = StockDataExtractor()
    extracted_data = extractor.process_input(state)

    return extracted_data

graph.add_node("entity_extraction", entity_extraction_node)

# **✅ Step 2: Data Fetching Node**
def data_fetching_node(state: EntityExtractOutput) -> DataFetchOutput:
    logger.info("Running Data Fetching Node...")
    fetcher = HistoricalDataFetcher()
    data_fetch_output = fetcher.fetch_and_store_historical_data(
        EntityExtractOutput(**state["entity_extract"])
    )
    return data_fetch_output

graph.add_node("data_fetching", data_fetching_node)
'''
# **✅ Step 3: Training Node**
def training_node(state: StockPredictionState) -> Dict:
    logger.info("Running Training Node...")
    trainer = StockDataTrainer()
    final_state = trainer.execute_training(DataFetchOutput(**state["data_fetch_output"]))

    return {"final_state": final_state}

graph.add_node("training", training_node)

# **✅ Step 4: Prediction Node**
def prediction_node(state: StockPredictionState) -> Dict:
    logger.info("Running Prediction Node...")
    predictor = StockPredictor(FinalPredictionState(**state["final_state"]))

    stock_request = state["final_state"]["entity_extract"]["entity_extract"]["stock_prediction"]
    stock_request_obj = StockPredictionRequest(**stock_request)

    prediction_output = predictor.make_predictions(stock_request_obj)

    return {"prediction": prediction_output}

graph.add_node("prediction", prediction_node)
'''
# **✅ Define Execution Order**
graph.add_edge(START, "entity_extraction")
graph.add_edge("entity_extraction", "data_fetching")
#graph.add_edge("data_fetching", "training")
#graph.add_edge("training", "prediction")
#graph.add_edge("prediction", END)

# **✅ Compile the Graph**
workflow = graph.compile()

# **✅ Running the Workflow**
state = workflow.invoke({
    "user_input": "quiero saber el precio de IBM de manera mensual para enero de 2025"
})

print(state)  # Output the final workflow state


2025-02-08 20:05:37,518 - StockPredictionWorkflow - INFO - Running Entity Extraction Node...
2025-02-08 20:05:37,518 - StockPredictionWorkflow - INFO - Running Entity Extraction Node...
2025-02-08 20:05:37,518 - StockPredictionWorkflow - INFO - Running Entity Extraction Node...
2025-02-08 20:05:37,518 - StockPredictionWorkflow - INFO - Running Entity Extraction Node...
2025-02-08 20:05:37,518 - StockPredictionWorkflow - INFO - Running Entity Extraction Node...
2025-02-08 20:05:37,520 - StockDataExtractor - INFO - Initializing StockDataExtractor with model 'mistral:7b'
2025-02-08 20:05:37,520 - StockDataExtractor - INFO - Initializing StockDataExtractor with model 'mistral:7b'
2025-02-08 20:05:37,520 - StockDataExtractor - INFO - Initializing StockDataExtractor with model 'mistral:7b'
2025-02-08 20:05:37,520 - StockDataExtractor - INFO - Initializing StockDataExtractor with model 'mistral:7b'
2025-02-08 20:05:37,520 - StockDataExtractor - INFO - Initializing StockDataExtractor with mode

ValidationError: 1 validation error for EntityExtractOutput
stock_prediction
  Field required [type=missing, input_value={'user_input': 'quiero sa...ual para enero de 2025'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing